<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex01noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML2022 ex01noteA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/Data-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?Data/2021


## 準備

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりもできます．
Python は，データ分析や機械学習・人工知能分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．ですので，以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行していってね．

In [ ]:
# 科学技術計算のライブラリ NumPy のモジュールを np という名前で使えるようにする
import numpy as np

# グラフを描くためのライブラリ matplotlib の pyplot を plt という名前でインポート
import matplotlib.pyplot as plt

# コンピュータビジョン・画像処理のためのライブラリ OpenCV のモジュール cv2 をインポート
import cv2

# 深層学習のためのライブラリ Keras のいろいろ
#import keras
from tensorflow import keras
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image

# 時間計測のためのほげ
import datetime

# notebook 上で画像を表示するためのほげ
from IPython.display import display, Image

### 画像表示用の関数
#
def displayImage(img):
    
    if type(img) is np.ndarray:               # 渡されたのが np.ndarray だった（OpenCVの画像はこの形式)
        rv, buf = cv2.imencode('.png', img)  # PNG形式のバイト列に変換
        if rv:
            display(Image(data = buf.tobytes()))   # 変換できたらバイト列を渡して表示
            return
    elif type(img) is str:                         # 渡されたのが文字列だった
        display(Image(filename = img))
        return
    
    print('displayImage: error')

----
## 多変量解析や機械学習に関するデモ
----

----
### クラスター分析／クラスタリング

**クラスター分析**／**クラスタリング** は，多変量データをクラスタ分けする（似たもの同士でグループに分ける）ための手法です．

ここでは，「K-means法（K-平均法）」と呼ばれるアルゴリズムを使って，猫の画像をクラスタリングしてみましょう．まずは，データやプログラムの準備を．以下のセルたちを順次実行してください．



(1) 猫画像データを入手します．

In [ ]:
### 猫画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/cat131.npz
cat = np.load('cat131.npz')['cat131']

(2) 学習などのための関数を定義します．このセルは関数を定義するだけなので，実行しても何も出力されないのが正常です．

In [ ]:
### セントロイドの初期化
#
def initCentroid(X, K):
    idx = np.random.permutation(X.shape[0])[:K]
    return X[idx, :]

### 最も近いセントロイドの番号を求める
#
def compIndex(X, centroid, index):
    for i, x in enumerate(X):
        d2 = np.sum((x[np.newaxis, :] - centroid )**2, axis = 1)
        index[i] = np.argmin(d2)

### セントロイドを求める
#
def compCentroid(X, centroid, index):
    for ik in range(centroid.shape[0]):
        centroid[ik] = np.mean(X[index == ik, :], axis = 0)

### コストおよびクラスタごとの所属データ数を求める
#
def compCost(X, centroid, index):
    cost = 0.0
    Nc = np.empty(centroid.shape[0], dtype = int)
    for ik in range(centroid.shape[0]):
        idx = index == ik
        cost += np.sum((X[idx, :] - centroid[ik, np.newaxis, :] )**2)
        Nc[ik] = np.sum(idx)
    return cost / (X.shape[0] * X.shape[1]), Nc


(3) 可視化のための関数を定義します．

In [ ]:
###  データの最初の nx x ny 枚を可視化
#
def mosaicImage(dat, nx, ny, nrow = 64, ncol = 64, gap = 4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))
            
    return img


###  セントロイドとともにデータの最初の n 枚を可視化
#
def mosaicImage2(X, centroid, index, n, nrow = 64, ncol = 64, gap = 4):

    nx = n + 2
    ny = centroid.shape[0]
    
    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        ix = 0
        ltx = ix*(ncol + gap) + gap
        img[lty:lty+nrow, ltx:ltx+ncol] = centroid[iy, :].reshape((nrow, ncol))
        dat = X[index == iy, :]
        for ix in range(2, nx):
            jx = ix - 2
            if jx >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[jx, :].reshape((nrow, ncol))
            
    return img


(4) 入手した猫画像の一部を表示してみましょう．

In [ ]:
print('### 猫画像 ###')
print('データ数 x 次元数 = {0[0]} x {0[1]}'.format(cat.shape))

# 最初の 20 枚を表示 
img = mosaicImage(cat, 5, 4)
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()  

(5) クラスタリングを実行して結果を可視化しましょう．

In [ ]:
### クラスタリング
#
X = cat   # 猫画像データを使用
K = 2       # クラスタ数
nitr = 10   # 繰り返し回数
N = X.shape[0]  # 学習データ数

print('# 学習回数  コスト  クラスタ毎の所属データ数')

# 初期化
centroid = initCentroid(X, K)
index = np.empty(N, dtype = int)
compIndex(X, centroid, index)
cost, Nc = compCost(X, centroid, index)
print('{0}  {1:.2f}     {2}'.format(0,  cost, Nc))

start = datetime.datetime.now()

# 学習の繰り返し
for i in range(1, nitr+1):  
    compCentroid(X, centroid, index)   # セントロイドを求める（資料2/6 step2）
    compIndex(X, centroid, index)         # 各データをクラスタへ割り当てる（同 step3）
    cost, Nc = compCost(X, centroid, index)
    print('{0}  {1:.2f}     {2}'.format(i,  cost, Nc))

print('# 経過時間:', datetime.datetime.now() - start)

# クラスタ毎のセントロイドと所属データを可視化
#     各行が一つのクラスタに対応，左端がセントロイド，右の画像はそのクラスタに所属する画像の一部
img = mosaicImage2(cat, centroid, index, 8)
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(img, cmap = 'gray')
#cv2.imwrite('{0}_K{1:03d}.png'.format(ID, K), img)  # 表示している画像をファイルに保存
plt.show()
s = '''
クラスタ毎のセントロイドと所属データを可視化
　各行が一つのクラスタに対応，左端がセントロイド，右の画像はそのクラスタに所属する画像の一部
'''
print(s)

**★★★ 結果を観察しよう＆やってみよう★★★**

上記では，猫画像を K = 2 の K-means法でクラスタリングする実験を行っています．左端に表示される画像は，K個のクラスタそれぞれに振り分けられたデータの平均です．

K をいろいろ変えて（上記セルの4行目の `K = 2` の `2` を変えて）実行し直して，結果を観察しよう．

ただし，データ数がそれほど多くないので，Kをあまり大きくすると空のクラスタができたりしてエラーになるかもしれません．また，K-meansクラスタリングの結果は初期値に依存するので，実行するたびに結果が変わります．

----
### ロジスティック回帰やニューラルネットによる手書き数字画像の識別

MNIST と呼ばれる手書き数字画像のデータを使って識別の実験をしてみよう．

参考:  http://yann.lecun.com/exdb/mnist/


#### データの準備

(1) データの準備その1

In [ ]:
# 上記サイトから 4 つのファイルを入手し， gunzip
! for fn in train-images-idx3-ubyte t10k-images-idx3-ubyte train-labels-idx1-ubyte t10k-labels-idx1-ubyte; do if [ ! -e ${fn} ]; then wget -nc http://yann.lecun.com/exdb/mnist/${fn}.gz ; gunzip ${fn}.gz; fi; done
! ls -l

# MNIST のためのクラスの定義

import struct
import os
import numpy as np

class MNIST:
    
    def __init__(self):

        fnImageL = 'train-images-idx3-ubyte'
        fnImageT = 't10k-images-idx3-ubyte'
        fnLabelL = 'train-labels-idx1-ubyte'
        fnLabelT = 't10k-labels-idx1-ubyte'
    
        if not os.path.exists(fnImageL) or not os.path.exists(fnImageT) or not os.path.exists(fnLabelL) or not os.path.exists(fnLabelT):
            print('Please get the MNIST files first.')
            return
       
        self.fnImage = {'L': fnImageL, 'T': fnImageT}
        self.fnLabel  = {'L': fnLabelL, 'T': fnLabelT}
        self.nrow = 28
        self.ncol = 28
        self.nclass = 10
        
        
    def getLabel( self, LT ):
        
        return _readLabel( self.fnLabel[LT] )
 
 
    def getImage( self, LT ):
        
        return _readImage( self.fnImage[LT] )
 
 
##### reading the label file
#
def _readLabel( fnLabel ):
 
    f = open( fnLabel, 'rb' )
 
    ### header (two 4B integers, magic number(2049) & number of items)
    #
    header = f.read( 8 )
    mn, num = struct.unpack( '>2i', header )  # MSB first (bigendian)
    assert mn == 2049
    #print mn, num
 
    ### labels (unsigned byte)
    #
    label = np.array( struct.unpack( '>%dB' % num, f.read() ), dtype = int )
 
    f.close()
 
    return label

 
##### reading the image file
#
def _readImage( fnImage ):
 
    f = open( fnImage, 'rb' )
 
    ### header (four 4B integers, magic number(2051), #images, #rows, and #cols
    #
    header = f.read( 16 )
    mn, num, nrow, ncol = struct.unpack( '>4i', header ) # MSB first (bigendian)
    assert mn == 2051
    #print mn, num, nrow, ncol
 
    ### pixels (unsigned byte)
    #
    npixel = ncol * nrow
    #pixel = np.empty( ( num, npixel ), dtype = int )
    #pixel = np.empty( ( num, npixel ), dtype = np.int32 )
    pixel = np.empty( ( num, npixel ) )
    for i in range( num ):
        buf = struct.unpack( '>%dB' % npixel, f.read( npixel ) )
        pixel[i, :] = np.asarray( buf )
 
    f.close()
 
    return pixel


(2) データの準備その2

In [ ]:
# MNIST の学習データ
mn = MNIST()
datL = mn.getImage('L')
labL = mn.getLabel('L')
NL = datL.shape[0]
print('# 学習データのデータ数: ', NL)
D = mn.nrow * mn.ncol
K = 10

# 学習データの最初の50個を可視化してみる
img = mosaicImage(datL[:50, :], 10, 5, nrow=28, ncol=28)
plt.figure(figsize=(8,8))
plt.axis('off')
plt.imshow(img, cmap = 'gray')

# これらの正解ラベル
for i in range(5):
    print(labL[i*10:((i+1)*10)])

このデータセットでは，1つの手書き数字画像は 28 x 28 画素ありますので，1つのデータを $784 (=28\times 28)$ 次元ベクトルとして扱うことになります．
画像の上に並んだ数字 `5 0 4 1 9 ...` は，これらの画像の手書き数字に対応する実際の数（正解）を表しています．

#### ロジスティック回帰とニューラルネットによる識別

前回ちょこっと説明した「ロジスティック回帰」は，与えられたデータを2通りに分類するものでしたが，よりたくさんの分類ができるように拡張できます．ここでは，784次元ベクトル6万個から成る手書き数字画像データを，0 から 9 の10通りに分類させてみましょう．

また，最近のいわゆる人工知能では，「ニューラルネット」という機械学習の仕組みがよく使われます（「深層学習 (deep learning)」ともいいます）．
以下では，簡単なニューラルネットの例でも実験できるようにしてあります．

(3) ネットワークモデルと学習アルゴリズムを定義

In [ ]:
### ネットワークを定義する関数
#
def makeNetwork(model_type, D, K, H1 = 256, H2 = 128):
    
    network = keras.models.Sequential()
    
    if model_type == 1:  # ロジスティック回帰
        print(f'# ロジスティック回帰やで．入力は{D}次元，出力は{K}次元ですわ．')
        network.add(keras.layers.Dense(K, input_shape = (D,)))
        network.add(keras.layers.Activation('softmax'))
    elif model_type == 2:  #  2層 MLP
        print(f'# 2層MLPでおま．入力は{D}次元，隠れ層のニューロン数は {H1}，出力のニューロン数は {K} でっせ．')
        network.add(keras.layers.Dense(H1, input_shape = (D,)))
        network.add(keras.layers.Activation('relu'))
        network.add(keras.layers.Dense(K))
        network.add(keras.layers.Activation('softmax'))
    elif model_type == 3:  #  3層 MLP
        print(f'# 3層MLPでおま．入力は{D}次元，ニューロン数は，(1つ目の隠れ層) - (2つ目の隠れ層) - (出力) の順に {H1} - {H2} - {K} でんがなまんがな．')
        network.add(keras.layers.Dense(H1, input_shape = (D,)))
        network.add(keras.layers.Activation('relu'))
        network.add(keras.layers.Dense(H2))
        network.add(keras.layers.Activation('relu'))
        network.add(keras.layers.Dense(K))
        network.add(keras.layers.Activation('softmax'))
    else:
        exit('makeNetwork: model_type error')

    # パラメータ最適化法の設定．確率的勾配降下法(Stochastic Gradient Descent)を用いる
    #optimizer = keras.optimizers.SGD(lr = 0.1, momentum = 0.9)
    optimizer = keras.optimizers.SGD(learning_rate = 0.1, momentum = 0.9)

    network.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return network

(4) 学習を実行し，テストデータ（学習に使っていない未知のデータ）で検証します．

In [ ]:
### ネットワークの作成
#
network = makeNetwork(1, D, K)  # 第1引数が 1 だとロジスティック回帰， 2 だと2層MLP， 3 だと3層MLP

### 学習
#
batchsize = 128
XL = datL / 255
YL = keras.utils.to_categorical(labL, num_classes = K)

print('# 学習回数  コスト  誤識別率[%]')
start = datetime.datetime.now()

for i in range(10000+1):  # 学習の繰り返し

    if (i < 500 and i % 100 == 0) or (i % 500 == 0):
        lossL, accL = network.evaluate(XL, YL, batch_size = batchsize, verbose = 0)  # 評価
        print('{}          {:.3f}          {:.2f}'.format(i, lossL, (1-accL)*100))
    
    ii = np.random.randint(XL.shape[0], size = batchsize)
    network.train_on_batch(XL[ii], YL[ii])     # 学習

print('# 経過時間:', datetime.datetime.now() - start)

lossL, accL = network.evaluate(XL, YL, batch_size = batchsize, verbose = 0)
print('# 学習データに対するコストと誤識別率[%]: {:.3f}   {:.2f}'.format(lossL, (1-accL)*100))

### テストデータの準備
#
datT = mn.getImage('T')
labT = mn.getLabel('T')
NT = datT.shape[0]
print('# テストデータのデータ数: ', NT)

### テスト
#
XT = datT / 255
YT = keras.utils.to_categorical(labT, num_classes = K)
lossT, accT = network.evaluate(XT, YT, batch_size = batchsize, verbose = 0)
print('# テストデータに対するコストと誤識別率[%]: {:.3f}   {:.2f}'.format(lossT, (1-accT)*100))

**★★★ 結果を観察しよう＆やってみよう★★★**

上記の「誤識別率」というのは，与えられたデータのうち，識別を間違えた（正解と違う数字と判定した）画像の割合です．「学習データ」は学習に使ったデータ，「テストデータ」は学習時に見たことのない未知のデータです．ロジスティック回帰だと，テストデータの8%くらいは間違えます．

上記のセルの3行目の
```
network = makeNetwork(1, D, K)  # 第1引数が 1 だとロジスティック回帰， 2 だと2層MLP， 3 だと3層MLP
```
というところを，コメントにしたがって書き換えると，ニューラルネットを使った実験もできます．実行して，結果がどう変わるか見てみましょう．ニューラルネットの学習は，ロジスティック回帰より時間がかかります．

----
### 1000種類の画像を認識するニューラルネットを動かしてみよう

(1) 画像の入手

In [ ]:
# 自分で画像をアップロードして識別させてみる場合は，以下を False に
useUni3 = True

if useUni3:
    # うに様の画像を入手
    ! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/uni3.png
else:
    # テストに使う画像をアップロード
    from google.colab import files
    uploaded = files.upload()

!ls -l

上記が正しく実行できれば，最後の行に
```
-rw-r--r-- 1 root root   227576 May  3  2018 uni3.png
```
のように，`uni3.png` というファイル名があるはずです．


(2) 画像を表示

In [ ]:
# 画像のファイル名を指定
if useUni3:
    fn = 'uni3.png'
else:
    fn = 'hoge.jpg'   ### 自分でアップロードした画像を使う場合は，ここを修正

# 入手した画像を読み込む
imgRaw = cv2.imread(fn)

# 画像を表示
displayImage(imgRaw)
print(imgRaw.shape)

(3) 学習済みのニューラルネットのパラメータを入手

In [ ]:
# VGG16 ネットワークモデルを作成
#   はじめて実行する時は，ネットワークパラメータをダウンロードするのに少し時間がかかる
model = VGG16(weights = 'imagenet')

(4) ニューラルネットに識別させる

In [ ]:
# 画像を読み込み．入力は 224x224 に大きさを調整される
img = image.load_img(fn, target_size = (224, 224))

# 前処理
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

# ネットワークの出力を求める
preds = model.predict(x)

# 画像を表示
plt.axis('off')
plt.imshow(img)
plt.show()

# 5位までの結果を表示（softmax出力の値 = 確率が高い方5つ）
print()
out = decode_predictions(preds, top=5)[0]
for i, item in enumerate(out):
    print('{1}位 {0[0]}  {0[1]}  {0[2]:.5f}'.format(item, i+1))

**★★★ 結果を観察しよう ★★★**

この実験で用いているニューラルネットは，120万枚の画像を学習データとして，1000種類の物体・画像を識別するよう学習したものです．この学習の過程は，高性能なPCでも1週間くらいかかるので，ここではすでに学習済みのニューラルネットを利用しています．

この設定では，「ネコ」は一つのクラスではなく，複数に分かれてます．

weasel 等の日本語訳は以下をどぞ．上記の右端の値は，ニューラルネットの出力そのものです．入力画像がそのクラスのものである確率の推定値になってます．そのクラスであることの確信度，とも言えます．

```
weasel: イタチ
lynx: ヤマネコ
black-footed_ferret: クロアシイタチ
Egyptian_cat: エジプト産のネコ，エジプシャンマウ(?)
polecat: スカンク(?)
```


**★★★ やってみよう ★★★**

次のようにすると，自分で用意した画像を識別させてみることができます．
やってみよう．

1. 画像を入手．ファイル名が長かったり日本語が含まれてたりするとうまく扱えないことがあるので，簡単な名前に変更しておくとよい．
1. 「(1) 画像を入手」の以下の行（2行目）の `True` を `False` に書き換えてセルを実行し直す．
```
useUni3 = True
```
1. 画像をアップロードする．
1. 「(2) 画像を表示」の以下の行（5行目）の `hoge.png` を，自分がアップロードしたファイルの名前に変更する．
```
    fn = 'hoge.png'   ### 自分でアップロードした画像を使う場合は，ここを修正
```
1. 画像が表示されることを確認．
1. 「(4) ニューラルネットに識別させる」を実行

